# DML Diff-in-Diff


In [32]:
import numpy as np
import pandas as pd
import doubleml as dml
import multiprocessing

from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

## Repeated Outcomes

We will focus on the the DGP1 from [Sant'Anna and Zhao (2020)](https://arxiv.org/abs/1812.01723), see Section 4.1 and 4.2.

In [33]:
def f_reg(Z):
  res = 210 + 27.4*Z[:, 0] + 13.7*(Z[:, 1] + Z[:, 2] + Z[:, 3])
  return res

def f_ps(Z):
  res = 0.75*(-Z[:, 0] + 0.5*Z[:, 1] - 0.25*Z[:, 2] - 0.1*Z[:, 3])
  return res

def dgp(n=200, dgp=1):
  # features
  X = np.random.normal(loc=0, scale=1, size=[n,4])
  Z_tilde_1 = np.exp(0.5*X[:, 0])
  Z_tilde_2 = 10 + X[:, 1] / (1 + np.exp(X[:, 0]))
  Z_tilde_3 = (0.6 + X[:, 0]*X[:, 2]/25)**3
  Z_tilde_4 = (20 + X[:, 1] + X[:, 3])**2

  Z_tilde = np.column_stack((Z_tilde_1, Z_tilde_2, Z_tilde_3, Z_tilde_4))
  Z = (Z_tilde - np.mean(Z_tilde, axis=0)) / np.std(Z_tilde, axis=0)
  
  # error terms
  epsilon_0 = np.random.normal(loc=0, scale=1, size=n)
  epsilon_1 = np.random.normal(loc=0, scale=1, size=[n,2])
  if dgp == 1:
    features_ps = Z
    features_reg = Z
  elif dgp == 2:
    features_ps = X
    features_reg = Z
  elif dgp == 3:
    features_ps = Z
    features_reg = X
  elif dgp == 4:
    features_ps = X
    features_reg = X
  else:
    ValueError("DGP type not specified")

  # treatment and propensities
  p = np.exp(f_ps(features_ps)) / (1 + np.exp(f_ps(features_ps)))
  U = np.random.uniform(low=0, high=1, size=n)
  D = 1.0 * (p >= U)

  # potential outcomes
  nu =  np.random.normal(loc=D*f_reg(features_reg), scale=1, size=n)
  Y0 = f_reg(features_reg) + nu + epsilon_0
  Y1_d0 = 2*f_reg(features_reg) + nu + epsilon_1[:, 0]
  Y1_d1 = 2*f_reg(features_reg) + nu + epsilon_1[:, 1]
  Y1 = D*Y1_d1 + (1-D)*Y1_d0

  return Y0, Y1, D, Z, Y1_d0, Y1_d1


The ATT should be zero:

In [34]:
_, _, D, _, Y1_d0, Y1_d1 = dgp(int(10e+6))

ATT = np.mean(Y1_d1[D == 1] - Y1_d0[D == 1])
print(f'Observed ATT: {ATT}')

Observed ATT: 0.0006548815163379875


In [35]:
np.random.seed(42)
n_rep = 1000
n_obs = 1000
lambda_T = 0.5

datasets_ro = []
datasets_rcs = []
for i in range(n_rep):
    Y0, Y1, D, X, _, _ = dgp(n=n_obs)

    # datasets with repeated outcomes (ro)
    Y_diff = Y1 - Y0
    data_ro =  dml.DoubleMLData.from_arrays(X, Y_diff, D)  
    datasets_ro.append(data_ro)

    # datasets with repeated cross sections
    U_T = np.random.uniform(low=0, high=1, size=n_obs)
    T = 1.0 * (U_T <= lambda_T)
    Y_obs = T * Y1 + (1-T)*Y0
    data_rcs = dml.DoubleMLDIDData.from_arrays(X, Y_obs, D, T)
    datasets_rcs.append(data_rcs)

In [36]:
ml_g = RandomForestRegressor(n_estimators=200)
ml_m = RandomForestClassifier(n_estimators=200)

n_cores = multiprocessing.cpu_count()
print(f"Number of Cores: {n_cores}")
# to be process folds in parallel
cores_used = 5

Number of Cores: 8


In [37]:
# set core to repeated outcomes
score = "RO"

coefs = np.full((n_rep), np.nan)
ses = np.full((n_rep), np.nan)
cover = np.full((n_rep), np.nan)
ci_length = np.full((n_rep), np.nan)

for i_rep in range(n_rep):
    print(i_rep)
    obj_dml_data = datasets_ro[i_rep]
    dml_DiD = dml.DoubleMLDID(obj_dml_data, 
                              ml_g=clone(ml_g),
                              ml_m=clone(ml_m),
                              score=score,
                              n_folds=5)

    dml_DiD.fit(n_jobs_cv=cores_used)

    # save coefs and ses
    coefs[i_rep] = dml_DiD.coef
    ses[i_rep] = dml_DiD.se

    # calculate confidence interval
    confint = dml_DiD.confint(level=0.95)
    cover[i_rep] = (confint['2.5 %'] < ATT) & (ATT < confint['97.5 %'])
    ci_length[i_rep] = confint['97.5 %'] - confint['2.5 %']

0
1


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


2
3


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


4
5
6
7
8
9
10
11


...

987
988


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


989


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


990
991
992
993
994
995
996
997
998
999


In [38]:
print(f'Avg. Bias: {coefs.mean()}')
print(f'Avg. asymptotic Variance: {np.square(ses).mean()}')
print(f'Coverage: {cover.mean()}')
print(f'Avg. CI length: {ci_length.mean()}')

Avg. Bias: 0.38370903062253947
Avg. asymptotic Variance: 3.453303936610878
Coverage: 0.988
Avg. CI length: 6.372696838360749


## Repeated Cross-sectional Data

Use the same data with coss-sectional observations

In [39]:
# set core to repeated outcomes
score = "RCS"

coefs = np.full((n_rep), np.nan)
ses = np.full((n_rep), np.nan)
cover = np.full((n_rep), np.nan)

for i_rep in range(n_rep):
    print(i_rep)
    obj_dml_data = datasets_rcs[i_rep]
    dml_DiD = dml.DoubleMLDID(obj_dml_data, 
                              ml_g=clone(ml_g),
                              ml_m=clone(ml_m),
                              score=score,
                              n_folds=5)

    dml_DiD.fit(n_jobs_cv=cores_used)

    # save coefs and ses
    coefs[i_rep] = dml_DiD.coef
    ses[i_rep] = dml_DiD.se

    # calculate confidence interval
    confint = dml_DiD.confint(level=0.95)
    cover[i_rep] = (confint['2.5 %'] < ATT) & (ATT < confint['97.5 %'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


14


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


15
16
17
18
19
20
21
22
23
24
25
26
27
28


...

387
388


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


389
390


c:\users\bam5698\github\doubleml-for-py\doubleml\_utils.py:252: UserWarning: Propensity predictions from learner RandomForestClassifier(n_estimators=200) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


391
392


In [ ]:
print(f'Coverage: {cover.mean()}')